In [ ]:
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.models as models
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torchvision import transforms as T
import gc 

In [ ]:
train_df = pd.read_csv("../input/plant-pathology-2021-fgvc8/train.csv")
#folder_path_256 = "../input/resized-plant2021/img_sz_256"
#data_paths_256 = os.listdir(folder_path_256)
train_df_cp = train_df.copy()
train_df_cp['label_list'] = train_df_cp['labels'].str.split(' ')

In [ ]:
def lbl_lgc(col,lbl_list):
    if col in lbl_list:
        res = 1 
    else:
        res = 0
    return res

In [ ]:
lbls = ['healthy','complex','rust','frog_eye_leaf_spot','powdery_mildew','scab']
for x in lbls:
    train_df_cp[x]=0


for x in lbls:
    train_df_cp[x] = np.vectorize(lbl_lgc)(x,train_df_cp['label_list'])

In [ ]:
class Plant_Dataset(Dataset):
  def __init__(self,folder_path,data_paths,data_df=train_df_cp,size=224,transforms=None, train=True):
    self.folder_path = folder_path
    self.data_paths = data_paths
    self.data_df = data_df
    self.transforms = transforms
    self.train = train
    self.size = size

  def __getitem__(self, idx):
    img_path = os.path.join(self.folder_path,self.data_paths[idx])
    image = cv2.imread(img_path)
    image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (self.size, self.size), interpolation=cv2.INTER_AREA)
    image = np.asarray(image)

    if self.train: #for train or validation data
     #label = self.data_df.loc[self.data_df['image']==self.data_paths[idx]].values[0][1]
      j = 0
      vector = [0]*6
      values = self.data_df.loc[self.data_df['image']==self.data_paths[idx]].values
      for i in range(3,9):
        num = values[0][i]
        vector[j] = num
        j = j+1
     
      vector=np.asarray(vector)
      
    
    if self.transforms:
       image = self.transforms(image=image)['image']

    if self.train:
     return image,vector #train or validation data
    else:
      return image,self.data_paths[idx] #test data


  def __len__(self):
    return len(self.data_paths)  

In [ ]:
def submission(images,predictions):
  str_preds = []
  img_names = []
  j=0  
  for vec in predictions:
    labels = []
    for i in range(len(vec[0])):
      if vec[0][i]==1:
       labels.append(lbls[i])
         
    l = ' '.join(labels)
    str_preds.append(l) 
    img_names.append(images[j][0])
    j += 1
    
  output = pd.DataFrame({'image': img_names, 'labels': str_preds})
  output.to_csv('submission.csv',index=False)     

In [ ]:
def model_test(model,test_loader):
 model.eval()
 images = []
 predictions = []   
 treshold = 0.5
 for i ,(img,img_name) in enumerate(test_loader):
    images.append(img_name)
    img = img.float()
    img = img.cuda()
    
    with torch.no_grad():
      output = torch.sigmoid(model(img)).float()
    
    output = torch.where(output > treshold, 1,0)
    predictions.append(output)
    
    del img
    del output
    
    gc.collect() 
    torch.cuda.empty_cache()
    
    
 return images,predictions

In [ ]:
def load_model(path,model,optimizer):
  checkpoint = torch.load(path)
  model.load_state_dict(checkpoint['model_state_dict'])
  optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
  epoch = checkpoint['epoch']
  train_loss = checkpoint['train_loss']
  val_loss = checkpoint['val_loss']
  train_acc = checkpoint['train_acc']
  val_acc = checkpoint['val_acc']
  return train_loss,val_loss,train_acc,val_acc,epoch

In [ ]:
test_path = "../input/plant-pathology-2021-fgvc8/test_images"
test_data_paths = os.listdir(test_path)

In [ ]:
gc.collect() 
torch.cuda.empty_cache()

In [ ]:
transforms = A.Compose([A.Normalize(),ToTensorV2()])

In [ ]:
test_data = Plant_Dataset(test_path,test_data_paths,transforms=transforms,train=False)
test_loader = DataLoader(test_data, batch_size=1, shuffle=False,num_workers=0)

In [ ]:
model = models.vgg19_bn(pretrained=False)
model.classifier[6] = nn.Linear(4096,6)
optimi = optim.AdamW(model.parameters(), lr=1e-4)
#load_model(model1_path,model,optimi)

In [ ]:
load_model('../input/model-5/vgg19bn_1',model,optimi)

In [ ]:
model.cuda()

In [ ]:
images,predictions = model_test(model,test_loader)
submission(images,predictions)